**Alunos: Caio Serpa, Eden Coelho e Roberto Sá**

#Configuração do ambiente, instalação do hadoop e spark

In [1]:
#instalar o Java 8 na maquina da sessão
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#setando jdk 8
#!sudo apt install openjdk-8-jdk
#!sudo update-alternatives --config java 

#spark
!wget -q https://ftp.unicamp.br/pub/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#extração do spark
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#findspark para facilitar na criação da sessão spark
!pip install -q findspark=
!pip install  pyspark==3.1.2

ERROR: Invalid requirement: 'findspark='
Hint: = is not a valid operator. Did you mean == ?
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!java -version

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


In [3]:
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

#variáveis de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

"""

os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 2G \
    --executor-memory 2G \
    pyspark-shell'

"""

print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME']) 

/usr/lib/jvm/java-8-openjdk-amd64
/content/spark-3.1.2-bin-hadoop3.2


In [4]:
!pip install findspark
import findspark

findspark.find()
findspark.init()

conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)
#sc.stop()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importação do conjunto de dados

Fonte dos dados: https://www.kaggle.com/datasets/najzeko/steam-reviews-2021?resource=download

Tamanho do arquivo: 8.17GB
Formato do arquivo: CSV

Conjunto de dados de cerca de 21 milhões de avaliações de usuários de cerca de 300 jogos diferentes no Steam. Obtido usando a API fornecida pelo Steam descrita na documentação do Steamworks

In [5]:
#recebendo os dados via api do google drive
from google.colab import drive 

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# Para quem usar Spark SQL
from pyspark.sql.functions import to_timestamp
spark = SparkSession \
    .builder \
    .getOrCreate()


#data_spark = spark.read.csv('gdrive/My Drive/Colab Notebooks/steam_reviews.csv', header=True, inferSchema=False)

In [7]:
from pyspark.sql.types import *

schema = StructType([
    StructField('_c0', IntegerType(), False),
    StructField('app_id',IntegerType(), False),
    StructField('app_name', StringType(),False),
    StructField('review_id',IntegerType(), False),
    StructField('language', StringType(),False),
    StructField('review', StringType(),False),
    StructField('timestamp_created', LongType(), False),
    StructField('timestamp_updated', LongType(), False),
    StructField('recommended', BooleanType(), False),
    StructField('votes_helpful',IntegerType(), False),
    StructField('votes_funny',IntegerType(), False),
    StructField('weighted_vote_score', FloatType(), False),
    StructField('comment_count',IntegerType(), False),
    StructField('steam_purchase', BooleanType(), False),
    StructField('received_for_free', BooleanType(), False),
    StructField('written_during_early_access', BooleanType(), False),
    StructField('author.steamid', LongType(), False),
    StructField('author.num_games_owned',IntegerType(), False),
    StructField('author.num_reviews',IntegerType(), False),
    StructField('author.playtime_forever', FloatType(), False),
    StructField('author.playtime_last_two_weeks', FloatType(), False),
    StructField('author.playtime_at_review', FloatType(), False),
    StructField('author.last_played', StringType(), False),
])


In [8]:
data_spark = spark.read.csv('gdrive/My Drive/Colab Notebooks/steam_reviews.csv', header=True, schema=schema)
#data_spark = spark.read.csv('gdrive/My Drive/Colab Notebooks/steam_reviews.csv', header=True, inferSchema=True)

In [9]:
data_spark.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- recommended: boolean (nullable = true)
 |-- votes_helpful: integer (nullable = true)
 |-- votes_funny: integer (nullable = true)
 |-- weighted_vote_score: float (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)
 |-- author.steamid: long (nullable = true)
 |-- author.num_games_owned: integer (nullable = true)
 |-- author.num_reviews: integer (nullable = true)
 |-- author.playtime_forever: float (nullable = true)
 |-- author.playtime_last_two_weeks: float (nullable = true)
 |-- author.pl

In [10]:
data_spark = data_spark.withColumnRenamed('_c0', 'id')\
            .withColumnRenamed('author.steamid', 'author_steamid')\
            .withColumnRenamed('author.num_games_owned', 'author_num_games_owned')\
            .withColumnRenamed('author.num_reviews', 'author_num_reviews')\
            .withColumnRenamed('author.playtime_forever', 'author_playtime_forever')\
            .withColumnRenamed('author.playtime_last_two_weeks', 'author_playtime_last_two_weeks')\
            .withColumnRenamed('author.playtime_at_review', 'author_playtime_at_review')\
            .withColumnRenamed('author.last_played', 'author_last_played')

In [11]:
data_spark.show()

+----+----------+--------------------+---------+--------+------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|  id|    app_id|            app_name|review_id|language|                              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+----+----------+--------------------+---------+--------+------------------------------------+-----------------+-----------------+-----------+----------

In [12]:
data_spark = data_spark.na.drop().cache()

In [13]:
data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|id |app_id|app_name                |review_id|language|review                                                     |timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+---+------+------------------------+---------+--------+----------------------------------------------------

In [14]:
#data_spark.count()

In [15]:
from pyspark.sql.functions import *

#função para remover o ponto e o zero da coluna autor_last_played
get_number_without_dot = udf(lambda s: s.split('.')[0], StringType())
data_spark = data_spark.withColumn('s_author_last_played', get_number_without_dot(data_spark.author_last_played)).cache()

In [16]:
data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+
|id |app_id|app_name                |review_id|language|review                                                     |timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|s_author_last_played|
+---+------+------------------------+---------+--------+----------

In [17]:
#Convertendo para longType() a coluna s_author_last played
data_spark = data_spark.withColumn('s_author_last_played',data_spark["s_author_last_played"].cast(LongType())).cache()

#Convertendo para data as seguintes colunas
data_spark  = data_spark.withColumn('s_author_last_played', from_unixtime(col('s_author_last_played')))\
          .withColumnRenamed('s_author_last_played','t_author_last_played')\
          .withColumn('t_timestamp_created', from_unixtime(col('timestamp_created')))\
          .withColumn('t_timestamp_updated', from_unixtime(col('timestamp_updated')))\
          .cache()

#dropando colunas
cols = ("timestamp_created","timestamp_updated","author_last_played")
data_spark = data_spark.drop(*cols).cache()

data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+--------------------+-------------------+-------------------+
|id |app_id|app_name                |review_id|language|review                                                     |recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|t_author_last_played|t_timestamp_created|t_timestamp_updated|
+---+------+------------------------+---------+--------+----------------------------------------